In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import statistics
import numpy as np

In [3]:
# Parameters
csv_filename = 'simulation_results/simulation_results_eil51_6000_800_cooling_types_a280.csv'
cooling_types_of_interest = ["logarithmic", "linear", "geometric"]

# Read the CSV
df = pd.read_csv(csv_filename)

# Convert 'Distances' from string to list of floats
df['Distances'] = df['Distances'].str.strip("[]").str.split(", ").apply(lambda x: list(map(float, x)))

# Filter for the desired cooling types
df = df[df['Cooling Type'].isin(cooling_types_of_interest)]

# Group by cooling type
grouped = df.groupby('Cooling Type')

for ct, group in grouped:
    # Extract the last distance from each run
    last_distances = group['Distances'].apply(lambda dist_list: dist_list[-1])
    
    # Minimum of the last distances across all runs
    min_last_distance = last_distances.min()
    
    # Mean and ±2σ range for the last distances
    mean_last = last_distances.mean()
    std_last = last_distances.std(ddof=1)  # sample std
    two_sigma = 2 * std_last
    lower = mean_last - two_sigma
    upper = mean_last + two_sigma
    
    print(f"Cooling Type: {ct}")
    print(f"  Minimum Last Distance Observed: {min_last_distance}")
    print(f"  Mean of Last Distances: {mean_last:.4f}")
    print(f"  2σ: {two_sigma:.4f}")
    print(f"  ±2σ range: [{lower:.4f}, {upper:.4f}]")
    print()


Cooling Type: geometric
  Minimum Last Distance Observed: 2632.5247620212335
  Mean of Last Distances: 2703.5499
  2σ: 76.7755
  ±2σ range: [2626.7744, 2780.3254]

Cooling Type: linear
  Minimum Last Distance Observed: 4828.575499169223
  Mean of Last Distances: 5140.3096
  2σ: 345.4632
  ±2σ range: [4794.8463, 5485.7728]

Cooling Type: logarithmic
  Minimum Last Distance Observed: 2807.459225884013
  Mean of Last Distances: 2937.5566
  2σ: 139.5537
  ±2σ range: [2798.0029, 3077.1103]



In [ ]:
# Parameters
csv_filename = 'simulation_results_eil51_6000_800_cooling_types_a280.csv'
cooling_types_of_interest = ["logarithmic","linear","geometric"]

# Read the CSV
df = pd.read_csv(csv_filename)

# Convert columns from strings to lists
df['Distances'] = df['Distances'].str.strip("[]").str.split(", ").apply(lambda x: list(map(float, x)))
df['Temperatures'] = df['Temperatures'].str.strip("[]").str.split(", ").apply(lambda x: list(map(float, x)))
df['Iterations'] = df['Iterations'].str.strip("[]").str.split(", ").apply(lambda x: list(map(int, x)))

# Filter for the desired cooling types
df = df[df['Cooling Type'].isin(cooling_types_of_interest)]

# Group data by cooling type
cooling_types = df['Cooling Type'].unique()

aggregated = {}
for ct in cooling_types:
    subset = df[df['Cooling Type'] == ct]
    if len(subset) == 0:
        continue
    
    # Convert runs into DataFrames
    dist_df = pd.DataFrame(subset['Distances'].tolist())    # rows = runs, cols = iterations
    temp_df = pd.DataFrame(subset['Temperatures'].tolist()) # rows = runs, cols = iterations
    iter_df = pd.DataFrame(subset['Iterations'].tolist())   # rows = runs, cols = iterations

    n_runs = len(subset)
    
    # Compute mean and ±2σ for distances
    dist_mean = dist_df.mean(axis=0)
    dist_std = dist_df.std(axis=0)
    dist_lower = dist_mean - 2 * dist_std
    dist_upper = dist_mean + 2 * dist_std

    # Since temperature is not stochastic and the same in each run,
    # just take the temperature from the first run
    temp = temp_df.iloc[0]

    # Compute max iterations from the first run (assuming all runs have similar length)
    max_iterations = iter_df.apply(lambda x: x.iloc[-1], axis=1).iloc[0]

    aggregated[ct] = {
        'dist_mean': dist_mean,
        'dist_lower': dist_lower,
        'dist_upper': dist_upper,
        'temp': temp,
        'iterations': range(len(dist_mean)),
        'max_iterations': max_iterations,
        'n_runs': n_runs
    }

# Create a figure with two subplots stacked vertically
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8), dpi=300)

# Plot Distance vs Temperature on the top subplot (ax1)
for ct, data in aggregated.items():
    ax1.plot(data['temp'], data['dist_mean'], label=f'{ct} cooling')
    ax1.fill_between(data['temp'], data['dist_lower'], data['dist_upper'], alpha=0.4)
ax1.set_xlabel('Temperature')
ax1.set_ylabel('Distance')
ax1.set_xscale('log')  
ax1.invert_xaxis()  # Reverse x-axis for decreasing temperature
ax1.legend()
ax1.grid(True)

# Plot Distance vs Iteration on the bottom subplot (ax2)
for ct, data in aggregated.items():
    it = data['iterations']
    ax2.plot(it, data['dist_mean'], label=f'{ct} cooling')
    ax2.fill_between(it, data['dist_lower'], data['dist_upper'], alpha=0.4)
ax2.set_xlabel('Iterations')
ax2.set_ylabel('Distance')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Zoom in on the last 5 degrees of temperature: Separate subplots for each cooling type
num_ct = len(aggregated)
fig, axes = plt.subplots(nrows=1, ncols=num_ct, figsize=(3 * num_ct, 4), dpi=300)

if num_ct == 1:
    # If there's only one cooling type, just use axes directly
    axes = [axes]

for ax, (ct, data) in zip(axes, aggregated.items()):
    temp_mean = data['temp']
    dist_mean = data['dist_mean']
    dist_lower = data['dist_lower']
    dist_upper = data['dist_upper']

    # Identify final temperature (lowest temperature)
    final_temp = temp_mean.iloc[-1]
    # Define cutoff as final_temp + 5
    cutoff_temp = final_temp + 5

    # Get indices where temperature <= cutoff_temp
    indices = [i for i, t in enumerate(temp_mean) if t <= cutoff_temp]

    if len(indices) > 0:
        start_idx = indices[0]
        # Plot only this section
        ax.plot(temp_mean[start_idx:], dist_mean[start_idx:], label=f'{ct} Cooling')
        ax.fill_between(temp_mean[start_idx:], dist_lower[start_idx:], dist_upper[start_idx:], alpha=0.2)

    ax.set_title(f'{ct}')
    ax.set_xlabel('Temperature')
    ax.set_ylabel('Distance')
    ax.set_xscale('log')
    ax.invert_xaxis()
    ax.grid(True)
    #ax.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Zoom in on last 100 iterations: Separate subplots for each cooling type
num_ct = len(aggregated)
fig, axes = plt.subplots(nrows=1, ncols=num_ct, figsize=(5 * num_ct, 6))

if num_ct == 1:
    # If there's only one cooling type, wrap it in a list
    axes = [axes]

for ax, (ct, data) in zip(axes, aggregated.items()):
    dist_mean = data['dist_mean']
    dist_lower = data['dist_lower']
    dist_upper = data['dist_upper']
    iterations = list(data['iterations'])

    # Determine the starting index for the last 100 iterations
    start_idx = max(0, len(dist_mean)-100)

    # Plot the last 100 iterations
    ax.plot(iterations[start_idx:], dist_mean[start_idx:], label=f'{ct} Cooling')
    ax.fill_between(iterations[start_idx:], dist_lower[start_idx:], dist_upper[start_idx:], alpha=0.2)

    ax.set_title(f'{ct} Cooling (Last 100 Iterations)')
    ax.set_xlabel('Iterations')
    ax.set_ylabel('Distance')
    ax.grid(True)
    ax.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Parameters
csv_filename = 'simulation_results_eil51_6000_800_cooling_types_a280.csv'
cooling_types_of_interest = ["geometric", "linear"]

# Read the CSV
df = pd.read_csv(csv_filename)

# Convert 'Distances', 'Temperatures', and 'Iterations' from string to list
df['Distances'] = df['Distances'].str.strip("[]").str.split(", ").apply(lambda x: list(map(float, x)))
df['Temperatures'] = df['Temperatures'].str.strip("[]").str.split(", ").apply(lambda x: list(map(float, x)))
df['Iterations'] = df['Iterations'].str.strip("[]").str.split(", ").apply(lambda x: list(map(int, x)))

# Filter for the desired cooling types
df = df[df['Cooling Type'].isin(cooling_types_of_interest)]

cooling_types = df['Cooling Type'].unique()

aggregated = {}
for ct in cooling_types:
    subset = df[df['Cooling Type'] == ct]
    if len(subset) == 0:
        continue

    # Convert all runs distances and temperatures into DataFrames
    dist_df = pd.DataFrame(subset['Distances'].tolist())       # rows = runs, cols = iterations
    temp_df = pd.DataFrame(subset['Temperatures'].tolist())    # rows = runs, cols = iterations

    # Compute per-run convergence (differences)
    convergence_df = dist_df.diff(axis=1).abs()  
    convergence_df.iloc[:,0] = 0.0  # Set the first difference to 0

    n_runs = len(subset)

    # Get temperature from one run (since temperature is not stochastic)
    temp = temp_df.iloc[0]

    # Compute mean and 2-sigma spread for convergence at each iteration
    convergence_mean = []
    convergence_lower = []
    convergence_upper = []

    for i in range(1, convergence_df.shape[1]):
        diffs_at_i = convergence_df.iloc[:, i].dropna().values
        mean_val = np.mean(diffs_at_i)
        std_val = np.std(diffs_at_i, ddof=1)  # sample std
        lower = mean_val - 2 * std_val
        upper = mean_val + 2 * std_val

        if lower < 0:
            lower = 0.0

        convergence_mean.append(mean_val)
        convergence_lower.append(lower)
        convergence_upper.append(upper)

    aggregated[ct] = {
        'temp': temp[1:],  # skip the first since i started from 1
        'conv_mean': convergence_mean,
        'conv_lower': convergence_lower,
        'conv_upper': convergence_upper,
        'n_runs': n_runs
    }

# Create a figure with two subplots: top (convergence vs temperature), bottom (convergence vs iteration)
fig, (ax_top, ax_bottom) = plt.subplots(nrows=2, ncols=1, figsize=(8,8), dpi=300, sharex=False)

# Top subplot: Convergence vs Temperature
for ct, data in aggregated.items():
    ax_top.plot(data['temp'], data['conv_mean'], label=f'{ct} cooling')
    ax_top.fill_between(data['temp'], data['conv_lower'], data['conv_upper'], alpha=0.2)

ax_top.set_xlabel('Temperature')
ax_top.set_ylabel(r'$|distance_i-distance_{i-1}|$')
ax_top.set_xscale('log')
ax_top.invert_xaxis()
ax_top.legend()
ax_top.grid(True)

# Bottom subplot: Convergence vs Iteration
for ct, data in aggregated.items():
    iterations = np.arange(1, len(data['conv_mean']) + 1)
    ax_bottom.plot(iterations, data['conv_mean'], label=f'{ct} cooling')
    ax_bottom.fill_between(iterations, data['conv_lower'], data['conv_upper'], alpha=0.2)

ax_bottom.set_xlabel('Iterations')
ax_bottom.set_ylabel(r'$|distance_i-distance_{i-1}|$')
ax_bottom.grid(True)
ax_bottom.legend()

plt.tight_layout()
plt.show()
